# canvas2mbgrader

In [1]:
from canvasapi import Canvas
import os
import tempfile
import glob

## STEP 1: Connect to Canvas LMS API

Create a Canvas access token:

1. Login to [canvas.ubc.ca](https://canvas.ubc.ca).
2. Navigate to *Account* > *Setting* > *Approved Integration* > *+ New Access Token* and follow the instructions.
3. Copy and paste your token into a text file called `token.txt` and save the file to the `mbgrader` folder (ie. the current folder).

Run the cell below to connect to the Canvas LMS API.

In [2]:
API_URL = 'https://ubc.instructure.com'
with open('token.txt','r') as f:
    API_KEY = f.read()
canvas = Canvas(API_URL, API_KEY)

me = canvas.get_user('self')
print('Welcome, {}!'.format(me.name))

Welcome, Patrick Walls!


## STEP 2: Select Canvas course and assignment

In [4]:
term = input('Enter term code (ie. 2022W1):')
print('Fetching Canvas courses ...')
courses = canvas.get_courses(include=['term'])
for course in courses:
    if course.term['name'] == term:
        print('ID:',course.id,'\tCourse Code:',course.course_code)
courseID = int(input('\nEnter Canvas course ID:'))
print('Fetching Canvas assignments ...')
course = canvas.get_course(courseID)
assignments = course.get_assignments()
for assignment in assignments:
    print('ID:',assignment.id,'\t Name:',assignment.name)
assignmentID = int(input('\nEnter Canvas assignment ID:'))
assignment = course.get_assignment(assignmentID)
assignment_name = input('Enter assignment folder name:')
destination = os.path.join('canvas',assignment_name)
os.makedirs(destination,exist_ok=True)
print('\n    Course:',course.course_code,'\n      Term:',term,'\nAssignment:',assignment.name,'\n    Folder:',destination)

Enter term code (ie. 2022W1): 2021W2


Fetching Canvas courses ...
ID: 85133 	Course Code: MATH 101 201 2021W2
ID: 92840 	Course Code: MATH 101 ALL 2021W2
ID: 85153 	Course Code: MATH 152 ALL 2021W2
ID: 85163 	Course Code: MATH 210 201 2021W2
ID: 85165 	Course Code: MATH 215 201/202 2021W2
ID: 85174 	Course Code: MATH 254 201 2021W2
ID: 85190 	Course Code: MATH 307 201 2021W2
ID: 85191 	Course Code: MATH 307 202 2021W2
ID: 85242 	Course Code: MECH 222 201 2021W2



Enter Canvas course ID: 85153


Fetching Canvas assignments ...
ID: 1119877 	 Name: Webwork Assignment00
ID: 1101970 	 Name: Webwork Assignment01
ID: 1101971 	 Name: Webwork Assignment02
ID: 1101972 	 Name: Webwork Assignment03
ID: 1101973 	 Name: Webwork Assignment04
ID: 1101963 	 Name: Webwork Assignment05
ID: 1101974 	 Name: Webwork Assignment06
ID: 1101975 	 Name: Webwork Assignment07
ID: 1101976 	 Name: Webwork Assignment08
ID: 1101977 	 Name: Webwork Assignment09
ID: 1101978 	 Name: Webwork Assignment10
ID: 1101979 	 Name: Webwork Assignment11 (for practice only)
ID: 1101952 	 Name: MATLAB Computer Lab 1
ID: 1101953 	 Name: MATLAB Computer Lab 2
ID: 1101954 	 Name: MATLAB Computer Lab 3
ID: 1101955 	 Name: MATLAB Computer Lab 4
ID: 1101956 	 Name: MATLAB Computer Lab 5
ID: 1101957 	 Name: MATLAB Computer Lab 6
ID: 1187025 	 Name: Exam 1B (4)
ID: 1187026 	 Name: Exam 1A
ID: 1202367 	 Name: Exam 2A
ID: 1202442 	 Name: Exam 2B (3)
ID: 1219488 	 Name: Final Exam



Enter Canvas assignment ID: 1101952
Enter assignment folder name: lab1



    Course: MATH 152 ALL 2021W2 
      Term: 2021W2 
Assignment: MATLAB Computer Lab 1 
    Folder: canvas/lab1


## STEP 3: Download submissions and move to `canvas` folder

Click the link output by the cell below to bulk download assignment submissions.

In [5]:
print(assignment.submissions_download_url)

https://ubc.instructure.com/courses/85153/assignments/1101952/submissions?zip=1


Move student files to the assignment folder:

In [6]:
print(destination)

canvas/lab1


Count total student files:

In [8]:
files = glob.glob(os.path.join(destination,'*'))
print('Found {} student files in {}.'.format(len(files),destination))

Found 930 student files in canvas/lab1.


## STEP 4: Create `canvasIDstudentID.csv`

Student files downloaded from Canvas contain Canvas user IDs in filenames. However, we need UBC student IDs for grading. Run the cell below to create the table `canvasIDstudentID.csv` which matches students Canvas IDs and UBC IDs.

In [9]:
students = course.get_users()
with open('canvasIDstudentID.csv','w') as f:
    for student in students:
        if student.sis_user_id:
            f.write('{},{}\n'.format(student.id,student.sis_user_id))

## STEP 5: Extract MATLAB data from .mat and .fig files

* Open MATLAB and navigate to `mbgrader` folder
* Run the script `canvas2mbgrader.m`

See `issues` folder to see MATLAB files that were not loaded.

## STEP 6: Run mbgrader

Open a new terminal and enter the commands:

```
conda activate mbgrader
python init_db.py
export FLASK_APP=app
flask run
```

Open a browser, navigate to [`http://127.0.0.1:5000`](http://127.0.0.1:5000/), grade the assignment and save the grades when complete.

## STEP 7: Upload grades and feedback

In [12]:
print('\n    Course:',course.course_code,'\n      Term:',term,'\nAssignment:',assignment.name,'\n    Folder:',os.path.join('submissions',assignment_name),'\n')
upload = input('Upload files to Canvas. Continue? [y]/n')

if upload in ['y','']:
    with open('canvasIDstudentID.csv') as f:
        lines = f.readlines()
        studentIDcanvasID = {}
        for line in lines:
            items = line.split(",")
            canvasID = int(items[0])
            studentID = int(items[1])
            studentIDcanvasID[studentID] = canvasID

    with open(os.path.join('grades',assignment_name + '.csv')) as f:
        lines = f.readlines()

    print('\nUploading for grades and feedback ...')
    for line in lines[1:]:
        items = line.split(',')
        student_id = int(items[0])
        canvas_id = studentIDcanvasID[student_id]
        try:
            submission = assignment.get_submission(canvas_id)
        except:
            print('Could not find assignment for {}'.format(canvas_id))
            continue
        print('{}'.format(student_id),end=' ')
        score = float(items[-1])
        submission.edit(submission={'posted_grade': score}
        f = tempfile.NamedTemporaryFile('w+')
        f.name = 'feedback.txt'
        source = os.path.join("feedback",assignment_name,'{}.txt'.format(student_id))
        with open(source,'r') as fsource:
            f.write(fsource.read())
        f.seek(0)
        submission.upload_comment(f)
        f.close()
    print('Done!')
else:
    print('Cancel upload.')


    Course: MATH 152 ALL 2021W2 
      Term: 2021W2 
Assignment: MATLAB Computer Lab 1 
    Folder: submissions/lab1 



Upload files to Canvas. Continue? [y]/n 



Uploading for grades and feedback ...
10175487 10200947 10252724 10289395 10348217 10442259 10553832 10835940 10840817 Done!
